In [1]:
import os
import sys
import time
import astro_ghost
# from astro_ghost.PS1QueryFunctions import getAllPostageStamps
# from astro_ghost.TNSQueryFunctions import getTNSSpectra
# from astro_ghost.NEDQueryFunctions import getNEDSpectra
from astro_ghost.ghostHelperFunctions import getTransientHosts, getGHOST
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
from datetime import datetime
import numpy as np

import tempfile

In [2]:
# From 106 available features from Kostya's lc_feature_extractor, use the 82 from SNAD Miner paper 
# R and g bands
feature_names_r_g = ['feature_amplitude_magn_r', 
                         'feature_anderson_darling_normal_magn_r',
                         'feature_beyond_1_std_magn_r', 
                         'feature_beyond_2_std_magn_r',
                         'feature_cusum_magn_r', 
                         #'feature_eta_e_magn_r',
                         'feature_inter_percentile_range_2_magn_r',
                         'feature_inter_percentile_range_10_magn_r',
                         'feature_inter_percentile_range_25_magn_r', 
                         'feature_kurtosis_magn_r',
                         'feature_linear_fit_slope_magn_r',
                         'feature_linear_fit_slope_sigma_magn_r',
                         #'feature_linear_fit_reduced_chi2_magn_r', 
                         'feature_linear_trend_magn_r',
                         'feature_linear_trend_sigma_magn_r',
                         'feature_magnitude_percentage_ratio_40_5_magn_r',
                         'feature_magnitude_percentage_ratio_20_5_magn_r',
                         #'feature_maximum_slope_magn_r',
                         'feature_mean_magn_r',
                         'feature_median_absolute_deviation_magn_r',
                         'feature_percent_amplitude_magn_r',
                         'feature_median_buffer_range_percentage_10_magn_r',
                         'feature_median_buffer_range_percentage_20_magn_r',
                         'feature_percent_difference_magnitude_percentile_5_magn_r',
                         'feature_percent_difference_magnitude_percentile_10_magn_r',
                         #'feature_period_0_magn_r',  # should be negated
                         'feature_period_s_to_n_0_magn_r',
                         #'feature_period_1_magn_r', 
                         'feature_period_s_to_n_1_magn_r',
                         #'feature_period_2_magn_r', 
                         'feature_period_s_to_n_2_magn_r',
                         #'feature_period_3_magn_r', 
                         'feature_period_s_to_n_3_magn_r',
                         #'feature_period_4_magn_r', 
                         'feature_period_s_to_n_4_magn_r',
                         #'feature_periodogram_amplitude_magn_r',
                         'feature_periodogram_beyond_2_std_magn_r',
                         'feature_periodogram_beyond_3_std_magn_r',
                         'feature_periodogram_standard_deviation_magn_r', 
                         #'feature_chi2_magn_r',
                         'feature_skew_magn_r', 
                         'feature_standard_deviation_magn_r',
                         'feature_stetson_k_magn_r',
                         'feature_weighted_mean_magn_r',
                         'feature_anderson_darling_normal_flux_r', 
                         'feature_cusum_flux_r',
                         #'feature_eta_e_flux_r', 
                         'feature_excess_variance_flux_r',
                         'feature_kurtosis_flux_r', 
                         'feature_mean_variance_flux_r',
                         #'feature_chi2_flux_r', 
                         'feature_skew_flux_r',
                         'feature_stetson_k_flux_r',

                         'feature_amplitude_magn_g', 
                         'feature_anderson_darling_normal_magn_g',
                         'feature_beyond_1_std_magn_g', 
                         'feature_beyond_2_std_magn_g',
                         'feature_cusum_magn_g', 
                         #'feature_eta_e_magn_g',
                         'feature_inter_percentile_range_2_magn_g',
                         'feature_inter_percentile_range_10_magn_g',
                         'feature_inter_percentile_range_25_magn_g', 
                         'feature_kurtosis_magn_g',
                         'feature_linear_fit_slope_magn_g',
                         'feature_linear_fit_slope_sigma_magn_g',
                         #'feature_linear_fit_reduced_chi2_magn_g', 
                         'feature_linear_trend_magn_g',
                         'feature_linear_trend_sigma_magn_g',
                         'feature_magnitude_percentage_ratio_40_5_magn_g',
                         'feature_magnitude_percentage_ratio_20_5_magn_g',
                         #'feature_maximum_slope_magn_g', 
                         'feature_mean_magn_g',
                         'feature_median_absolute_deviation_magn_g',
                         'feature_median_buffer_range_percentage_10_magn_g',
                         'feature_median_buffer_range_percentage_20_magn_g',
                         'feature_percent_amplitude_magn_g',
                         'feature_percent_difference_magnitude_percentile_5_magn_g',
                         'feature_percent_difference_magnitude_percentile_10_magn_g',
                         #'feature_period_0_magn_g',  # should be negated
                         'feature_period_s_to_n_0_magn_g',
                         #'feature_period_1_magn_g', 
                         'feature_period_s_to_n_1_magn_g',
                         #'feature_period_2_magn_g', 
                         'feature_period_s_to_n_2_magn_g',
                         #'feature_period_3_magn_g', 
                         'feature_period_s_to_n_3_magn_g',
                         #'feature_period_4_magn_g', 
                         'feature_period_s_to_n_4_magn_g',
                         #'feature_periodogram_amplitude_magn_g',
                         'feature_periodogram_beyond_2_std_magn_g',
                         'feature_periodogram_beyond_3_std_magn_g',
                         'feature_periodogram_standard_deviation_magn_g', 
                         #'feature_chi2_magn_g',
                         'feature_skew_magn_g', 
                         'feature_standard_deviation_magn_g',
                         'feature_stetson_k_magn_g', 
                         'feature_weighted_mean_magn_g',
                         'feature_anderson_darling_normal_flux_g', 
                         'feature_cusum_flux_g',
                         #'feature_eta_e_flux_g', 
                         'feature_excess_variance_flux_g',
                         'feature_kurtosis_flux_g', 
                         'feature_mean_variance_flux_g',
                         #'feature_chi2_flux_g', 
                         'feature_skew_flux_g',
                         'feature_stetson_k_flux_g'] 
    
ztf_id_and_features_r_g = ['Unnamed: 0'] + ['locus_id', 'ra', 'dec',\
                            'tags', 'catalogs', 'ztf_object_id', 'ztf_ssnamenr', 'num_alerts',\
                            'num_mag_values', 'oldest_alert_id', 'oldest_alert_magnitude',\
                            'oldest_alert_observation_time', 'newest_alert_id',\
                            'newest_alert_magnitude', 'newest_alert_observation_time',\
                            'brightest_alert_id', 'brightest_alert_magnitude', \
                            'brightest_alert_observation_time'] + feature_names_r_g +\
                        ['horizons_targetname', 'anomaly', 'anom_score', 'anomaly_score',\
                         'anomaly_mask', 'anomaly_type', 'is_corrected', 'vpdf_extreme_version',\
                         'vpdf_extreme_faint', 'vpdf_extreme_bright', 'locus_gal_l', 'locus_gal_b']

In [3]:
dataset_bank = pd.read_csv('../loci_dbs/loci_df_271688objects_cut_stars_and_gal_plane_transf.csv', 
                                   compression='gzip',
                                   usecols=lambda c: c in ztf_id_and_features_r_g,
                                  index_col='ztf_object_id')
        
# drop row if any feature values are nan
dataset_bank = dataset_bank[~dataset_bank[feature_names_r_g].isnull().any(axis=1)]

#mask = (dataset_bank['num_mag_values'] >= 40) #& (dataset_bank['num_mag_values'] <= 40)
#dataset_bank = dataset_bank[mask]
#dataset_bank.to_csv('./for_alex.csv')
dataset_bank

/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (7,125,129,130,131,132,133,134) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,locus_id,ra,dec,tags,catalogs,ztf_ssnamenr,num_alerts,num_mag_values,oldest_alert_id,...,anom_score,anomaly_score,anomaly_mask,anomaly_type,is_corrected,vpdf_extreme_version,vpdf_extreme_faint,vpdf_extreme_bright,locus_gal_l,locus_gal_b
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF20acpdyyq,6,ANT2020rajqu,33.976088,-20.258905,['lc_feature_extractor'],"['allwise', 'gaia_dr2', '2mass_psc', 'bright_g...",NaN,137,35,ztf_candidate:993481181415010007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197.881532,-69.513571
ZTF18acfvwwa,39,ANT2020pc6ko,35.639509,32.833982,['lc_feature_extractor'],"['allwise', 'gaia_dr2', '2mass_psc', 'bright_g...",NaN,659,88,ztf_candidate:943483942915015010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.200342,-26.246909
ZTF17aaaskvj,58,ANT2020rdpmm,28.015003,36.953217,['lc_feature_extractor'],"['allwise', 'csdr2', 'gaia_dr2', '2mass_psc', ...",NaN,1497,579,ztf_candidate:598425334815010002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.189482,-24.352953
ZTF17aaaskvj,59,ANT2018ftblk,28.015003,36.953216,"['lc_feature_extractor', 'high_snr']","['allwise', 'csdr2', 'gaia_dr2', '2mass_psc', ...",NaN,1504,583,ztf_candidate:598425334815010002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.189482,-24.352954
ZTF18aczbnhl,97,ANT2019dsk24,85.049145,-11.013302,['lc_feature_extractor'],"['allwise', 'gaia_dr2', '2mass_psc', 'bright_g...",NaN,105,12,ztf_candidate:718232301815010005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,214.961559,-20.751422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20aberawv,999927,ANT2020aebbk,292.910293,54.078978,"['lc_feature_extractor', 'dimmers']",[],NaN,235,96,ztf_candidate:1231482062015010023,...,NaN,0.417276,Fail,TBD,NaN,NaN,NaN,NaN,85.961850,16.149525
ZTF18acrvsjf,999938,ANT2019zf6fg,72.820139,1.073984,['lc_feature_extractor'],"['allwise', 'bright_guide_star_cat', '2mass_ps...",NaN,40,14,ztf_candidate:696399341515010050,...,NaN,NaN,NaN,TBD,NaN,NaN,NaN,NaN,197.132900,-25.906137
ZTF18acdqrna,999957,ANT2020aefhrvi,53.476865,77.496741,['lc_feature_extractor'],"['allwise', 'bright_guide_star_cat', '2mass_ps...",NaN,83,36,ztf_candidate:1353408024115015003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.425108,17.392307


In [ ]:
%%time

# Throw RA/DEC into ghost with just DLR method, gentle starcut
# Sets environ var to find ghost.csv
os.environ['GHOST_PATH'] = './host_info'
# Then don't use getGHOST(real=True, verbose=verbose)
getGHOST(real=True,verbose=False)

start_idx_l = list(np.arange(77200, len(dataset_bank), 100))
end_idx_l = list(np.arange(77300, len(dataset_bank)+100, 100))

for start, end in zip(start_idx_l, end_idx_l):
    start_idx, end_idx = start, end
    print("\nSTART:", start, "END:", end)

    dataset_bank_mini = dataset_bank.iloc[start_idx:end_idx]
    #print(dataset_bank_mini)

    snCoord = [SkyCoord(ra*u.deg, dec*u.deg, frame='icrs') for ra, dec in zip(dataset_bank_mini.ra, dataset_bank_mini.dec)]
    with tempfile.TemporaryDirectory() as tmp:
#        try: 
        hosts = getTransientHosts(snName=list(dataset_bank_mini.index), snCoord=snCoord, verbose=0, starcut='gentle', ascentMatch=False, savepath=tmp)
        # Save
        hosts.to_csv(f'./host_info/dataset_bank_hosts_start_idx:end_idx={start_idx}:{end_idx}.csv', index=False)
        print("Saved")
#        except: 
        #print(f"Throws 500 error. Continue!")
        #continue
        

GHOST database already exists in the install path!

START: 77200 END: 77300
Finding northern sources with Pan-starrs...
Processed 10 of 100 lines!
Processed 20 of 100 lines!
Processed 30 of 100 lines!
Processed 40 of 100 lines!
Processed 50 of 100 lines!
Processed 60 of 100 lines!
Processed 70 of 100 lines!
Processed 80 of 100 lines!
Processed 90 of 100 lines!
Processed 100 of 100 lines!


/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/astro_ghost/stellarLocus.py:64: RuntimeWarning: All-NaN slice encountered
  df.loc[i,"7DCD"] = np.nanmin(np.array(temp_7DCD_1val))


Found matches for 34.0% of events.
Saved

START: 77300 END: 77400
Finding northern sources with Pan-starrs...
Processed 10 of 100 lines!
Processed 20 of 100 lines!
Processed 30 of 100 lines!
Processed 40 of 100 lines!
Processed 50 of 100 lines!
Processed 60 of 100 lines!
Processed 70 of 100 lines!
Processed 80 of 100 lines!
Processed 90 of 100 lines!
Processed 100 of 100 lines!


/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/astro_ghost/stellarLocus.py:64: RuntimeWarning: All-NaN slice encountered
  df.loc[i,"7DCD"] = np.nanmin(np.array(temp_7DCD_1val))


Found matches for 27.0% of events.
Saved

START: 77400 END: 77500
Finding northern sources with Pan-starrs...
Processed 10 of 100 lines!
Processed 20 of 100 lines!
Processed 30 of 100 lines!
Processed 40 of 100 lines!
Processed 50 of 100 lines!
Processed 60 of 100 lines!
Processed 70 of 100 lines!
Processed 80 of 100 lines!
Processed 90 of 100 lines!
Processed 100 of 100 lines!


...

Found matches for 34.0% of events.
Saved

START: 77900 END: 78000
Finding northern sources with Pan-starrs...
Processed 10 of 100 lines!
Processed 20 of 100 lines!
Processed 30 of 100 lines!
Processed 40 of 100 lines!
Processed 50 of 100 lines!
Processed 60 of 100 lines!
Processed 70 of 100 lines!
Processed 80 of 100 lines!
Processed 90 of 100 lines!
Processed 100 of 100 lines!


/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/astro_ghost/stellarLocus.py:64: RuntimeWarning: All-NaN slice encountered
  df.loc[i,"7DCD"] = np.nanmin(np.array(temp_7DCD_1val))


Found matches for 28.0% of events.
Saved

START: 78000 END: 78100
Finding northern sources with Pan-starrs...
Processed 10 of 100 lines!
Processed 20 of 100 lines!
Processed 30 of 100 lines!
Processed 40 of 100 lines!
Processed 50 of 100 lines!
Processed 60 of 100 lines!
Processed 70 of 100 lines!
Processed 80 of 100 lines!
Processed 90 of 100 lines!
Processed 100 of 100 lines!


/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/astro_ghost/stellarLocus.py:64: RuntimeWarning: All-NaN slice encountered
  df.loc[i,"7DCD"] = np.nanmin(np.array(temp_7DCD_1val))


Found matches for 37.0% of events.
Saved

START: 78100 END: 78200
Finding northern sources with Pan-starrs...
Processed 10 of 100 lines!
Processed 20 of 100 lines!
Processed 30 of 100 lines!
Processed 40 of 100 lines!
Processed 50 of 100 lines!
Processed 60 of 100 lines!
Processed 70 of 100 lines!
Processed 80 of 100 lines!
Processed 90 of 100 lines!
Processed 100 of 100 lines!


In [5]:
astro_ghost.__version__

'0.2.1'

hosts.to_csv(f'./host_info/dataset_bank_hosts_start_idx:end_idx={start_idx}:{end_idx}.csv', index=False)
hosts

In [ ]:
# 100 objs -- 5 min (at 30% matches)